<a href="https://colab.research.google.com/github/AvellinaLeong/NHM-Nannofossil-Segmentation-Project/blob/main/02_morphometrics_on_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Morphometrics on train, test and val annotations

Morphometric parameters:


*   Major axis
*   Minor axis
*   Circularity
*   Ellipticity
*   Area


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Set script location to own development space
MY_DEVELOPMENT_SPACE = '/content/drive/MyDrive/development/avellina/'
import os
os.chdir(MY_DEVELOPMENT_SPACE)
!pwd
!ls

Mounted at /content/drive/
/content/drive/MyDrive/development/avellina
detectron2  Detectron2_notebooks  Mask-RCNN  Morphometrics_notebooks  output


In [ ]:
import os
import json
import csv
import cv2
import numpy as np
from skimage.measure import regionprops, label
from skimage.draw import polygon
from skimage.io import imread

In [ ]:
# File paths
datasets = {
    "train": {
        "annotations_path": "/content/drive/MyDrive/data/species_16/data/train/e_turriseiffelii_label_train_coco.json",
        "images_directory": "/content/drive/MyDrive/data/species_16/data/train"
    },
    "val": {
        "annotations_path": "/content/drive/MyDrive/data/species_16/data/val/e_turriseiffelii_label_val_coco.json",
        "images_directory": "/content/drive/MyDrive/data/species_16/data/val"
    },
    "test": {
        "annotations_path": "/content/drive/MyDrive/data/species_16/data/test/e_turriseiffelii_label_test_coco.json",
        "images_directory": "/content/drive/MyDrive/data/species_16/data/test"
    }
}

output_csv_path = "/content/drive/MyDrive/data/species_16/morphometric_output/morphometrics_all.csv"

In [ ]:
# Function to create a mask from COCO polygon annotations
def create_mask_from_annotation(image_shape, annotations):
    mask = np.zeros(image_shape, dtype=np.uint8)
    for annotation in annotations:
        polygons = [np.array(segment).reshape(-1, 2) for segment in annotation['segmentation']]
        for polygon_points in polygons:
            rr, cc = polygon(polygon_points[:, 1], polygon_points[:, 0], mask.shape)
            mask[rr, cc] = 1
    return mask

In [ ]:
# Open CSV file for writing
with open(output_csv_path, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["file_name", "class_name", "object_number", "major_axis", "minor_axis", "circularity", "ellipticity", "area"])

    for dataset_name, dataset_info in datasets.items():
        annotations_path = dataset_info["annotations_path"]
        images_directory = dataset_info["images_directory"]

        # Load COCO annotations
        with open(annotations_path, 'r') as file:
            coco_annotations = json.load(file)

        # Create a dictionary to map image IDs to file names
        image_id_to_filename = {image['id']: image['file_name'] for image in coco_annotations['images']}

        # Iterate over images in the COCO annotations
        for annotation in coco_annotations['annotations']:
            image_id = annotation['image_id']
            image_filename = image_id_to_filename[image_id]
            image_path = os.path.join(images_directory, image_filename)

            # Read image
            image = imread(image_path)

            # Create mask
            mask = create_mask_from_annotation(image.shape[:2], [annotation])

            # Label connected components
            labeled_mask = label(mask)

            # Check if the mask is empty
            if np.count_nonzero(labeled_mask) == 0:
                print(f"Warning: Image {image_filename} has an empty mask. Skipping.")
                continue

            props = regionprops(labeled_mask)

            scale = 0.0735

            for i, prop in enumerate(props):
                object_number = i + 1

                # Print out region properties for debugging
                print(f"Image: {image_filename}, Object {object_number}:")
                print(f"  - Major Axis Length: {prop.major_axis_length}")

                # Handle edge case where minor_axis_length might be very small or zero -- debug solution
                try:
                    minor_axis_length = prop.minor_axis_length
                    if np.isnan(minor_axis_length) or minor_axis_length <= 0:
                        print(f"Warning: Object {object_number} in image {image_filename} has invalid minor_axis_length ({minor_axis_length}). Skipping.")
                        continue
                except Exception as e:
                    print(f"Warning: Object {object_number} in image {image_filename} has error in minor_axis_length calculation: {e}. Skipping.")
                    continue

                print(f"  - Minor Axis Length: {minor_axis_length}")

                max_diameter = prop.major_axis_length * scale
                min_diameter = prop.minor_axis_length * scale

                # Circularity and Ellipticity
                circularity = np.sqrt((min_diameter * max_diameter) / (max_diameter ** 2))
                ellipticity = max_diameter / min_diameter

                # Area
                area = prop.area * (scale ** 2)

                class_name = 'e. turriseiffelii'

                # Save to CSV
                csvwriter.writerow([image_filename, class_name, object_number, max_diameter, min_diameter, circularity, ellipticity, area])

print("Morphometrics successfully saved to CSV file.")

Image: PM_NF_4999_03_26.jpeg, Object 1:
  - Major Axis Length: 68.1532975045294
  - Minor Axis Length: 48.34539830814738
Image: PM_NF_4999_14_44.jpeg, Object 1:
  - Major Axis Length: 61.98824386181804
  - Minor Axis Length: 43.980062766714845
Image: PM_NF_4999_20_11.jpeg, Object 1:
  - Major Axis Length: 102.85176504842383
  - Minor Axis Length: 69.0653520559596
Image: PM_NF_5053_02_8.jpeg, Object 1:
  - Major Axis Length: 78.81190239610314
  - Minor Axis Length: 56.00625157063443
Image: PM_NF_5054_02_22.jpeg, Object 1:
  - Major Axis Length: 93.39014140278348
  - Minor Axis Length: 61.687907183922576
Image: PM_NF_5065_11_40.jpeg, Object 1:
  - Major Axis Length: 79.47428481570222
  - Minor Axis Length: 56.091584761494474
Image: PM_NF_5065_12_45.jpeg, Object 1:
  - Major Axis Length: 67.99658658608931
  - Minor Axis Length: 49.64206459929532
Image: PM_NF_5065_25_37.jpeg, Object 1:
  - Major Axis Length: 90.23679105653846
  - Minor Axis Length: 65.49313908050858
Image: PM_NF_5225_10_7.